In [1]:
cd /content/drive/MyDrive/GenAI Project/

/content/drive/MyDrive/GenAI Project


In [ ]:
import pandas as pd

chapters_dsc = pd.read_csv('Dataset/Test/itihasa_chapters_desc.csv')

In [ ]:
chapters_dsc['Ch ID'] = chapters_dsc['Parva'].astype(str) + '.' + chapters_dsc['Chapter'].astype(str)
chapters_dsc['Full desc'] = chapters_dsc['Description'].fillna('') + '  ' + chapters_dsc['Detailed description'].fillna('')

In [ ]:
chapters_dsc

,S. No.,MNC_chapter,Volume,Parva,Chapter,Description,No Shlokas,Ch ID,Detailed description,Full desc
0,1,1,vol-i,1,1,Contents of subject,225.0,1.1,"The book's premise, a brief list of most of th...","Contents of subject The book's premise, a bri..."
1,2,2,vol-i,1,2,Brief introduction to the story of Mahabharata,337.0,1.2,"Description of Samantpanchak region, evidence ...",Brief introduction to the story of Mahabharata...
2,3,3,vol-i,1,3,"Curse of Janamejaya by Sarama, Story of the Ar...",177.0,1.3,"Saramaka's curse on Janamejaya, Janamejaya's s...","Curse of Janamejaya by Sarama, Story of the Ar..."
3,4,4,vol-i,1,4,Introduction of the story,11.0,1.4,NaN,Introduction of the story
4,5,5,vol-i,1,5,Dialogue between Puloma and Agni,31.0,1.5,Arrival of demon Puloma at Bhrigu's ashram and...,Dialogue between Puloma and Agni Arrival of d...
...,...,...,...,...,...,...,...,...,...,...
2105,2106,312,vol-ix,18,2,-Yudhishthira's solicitude for his brothers an...,48.0,18.2,-Yudhishthira's solicitude for his brothers an...,-Yudhishthira's solicitude for his brothers an...
2106,2107,313,vol-ix,18,3,"The advent of the deities, and their adulation...",39.0,18.3,"The advent of the deities, and their adulation...","The advent of the deities, and their adulation..."
2107,2108,314,vol-ix,18,4,Yudhishthira sees Govinda in his effulgent Bra...,21.0,18.4,Yudhishthira sees Govinda in his effulgent Bra...,Yudhishthira sees Govinda in his effulgent Bra...
2108,2109,315,vol-ix,18,5,"Janamejaya's query regarding the period ""The h...",55.0,18.5,"Janamejaya's query regarding the period ""The h...","Janamejaya's query regarding the period ""The h..."


In [ ]:
filtered_chapters_dsc = chapters_dsc[chapters_dsc['Detailed description'].notna()]
filtered_chapters_dsc = filtered_chapters_dsc[['Ch ID', 'Full desc']]


In [ ]:
filtered_chapters_dsc

,Ch ID,Full desc
0,1.1,"Contents of subject The book's premise, a bri..."
1,1.2,Brief introduction to the story of Mahabharata...
2,1.3,"Curse of Janamejaya by Sarama, Story of the Ar..."
4,1.5,Dialogue between Puloma and Agni Arrival of d...
5,1.6,"Birth of Chyavana, Bhrigu's curse on Agni Bir..."
...,...,...
2105,18.2,-Yudhishthira's solicitude for his brothers an...
2106,18.3,"The advent of the deities, and their adulation..."
2107,18.4,Yudhishthira sees Govinda in his effulgent Bra...
2108,18.5,"Janamejaya's query regarding the period ""The h..."


In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
import pandas as pd
from openai import OpenAI
import json
import time

# Initialize client (make sure your OPENAI_API_KEY is set)
client = OpenAI()

# Define prompt template
PROMPT_TEMPLATE = """
You are given a short description of a chapter from Mahabharata. Your task is to generate diverse, natural-sounding queries that a reader of Mahabharata might ask when trying to find this chapter using a search or retrieval system. The queries should reflect the main ideas and entities in the description but should not copy sentences directly. Generate 2 questions per chapter description Return in this format {{ ch_id: [ q1, q2] }}
Here are the chapter descriptions:
{chapter_texts}
"""

# Store results
results = {}
all_texts = {}
# Process 10 rows at a time
batch_size = 10

for i in range(0, len(filtered_chapters_dsc), batch_size):
  if i+batch_size < len(filtered_chapters_dsc):
      batch = filtered_chapters_dsc.iloc[i:i+batch_size]
      chapter_texts = "\n".join(
          [f"{row['Ch ID']}: {row['Full desc']}" for _, row in batch.iterrows()]
      )

      prompt = PROMPT_TEMPLATE.format(chapter_texts=chapter_texts)

      try:
          print(f"Processing rows {i} - {i+len(batch)-1}")
          response = client.chat.completions.create(
              model="gpt-4o-mini",  # or "gpt-4o" for higher quality
              messages=[
                  {"role": "system", "content": "You are an expert dataset generator for IR systems."},
                  {"role": "user", "content": prompt}
              ],
              temperature=0,
          )

          usage = response.usage
          prompt_tokens = usage.prompt_tokens
          completion_tokens = usage.completion_tokens

          print(f"✅ Batch {i}-{i+len(batch)-1}: | Prompt={prompt_tokens}, Completion={completion_tokens}")

          text = response.choices[0].message.content.strip('```').strip('json')

          try:
              batch_results = json.loads(text)
              results.update(batch_results)
          except Exception as e:
              print(f"{e}: JSON parse failed for batch starting at index {i}. Raw output:")
              # print(text)
              all_texts[i] = text
              continue

          # print(f"✅ Processed rows {i}–{i+len(batch)-1}")
          time.sleep(1)  # polite pause to avoid hitting rate limits

      except Exception as e:
          print(f"❌ Error in batch {i}: {e}")
          time.sleep(5)
          continue

# Save all results
with open("Dataset/Test/test_dataset.jsonl", "w", encoding="utf-8") as f:
    for item in results:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

# Optionally convert to CSV
output_df = pd.DataFrame(results)
output_df.T.to_csv("Dataset/Test/test_dataset.csv", encoding="utf-8")

print("🎉 Done! Generated IR dataset saved as ir_dataset.jsonl and ir_dataset.csv")


Processing rows 0 - 9
✅ Batch 0-9: | Prompt=534, Completion=451
Processing rows 10 - 19
✅ Batch 10-19: | Prompt=436, Completion=457
Processing rows 20 - 29
✅ Batch 20-29: | Prompt=487, Completion=476
Expecting property name enclosed in double quotes: line 2 column 3 (char 4): JSON parse failed for batch starting at index 20. Raw output:
Processing rows 30 - 39
✅ Batch 30-39: | Prompt=501, Completion=361
Expecting property name enclosed in double quotes: line 2 column 3 (char 4): JSON parse failed for batch starting at index 30. Raw output:
Processing rows 40 - 49
✅ Batch 40-49: | Prompt=471, Completion=366
Expecting property name enclosed in double quotes: line 2 column 3 (char 4): JSON parse failed for batch starting at index 40. Raw output:
Processing rows 50 - 59
✅ Batch 50-59: | Prompt=549, Completion=508
Processing rows 60 - 69
✅ Batch 60-69: | Prompt=467, Completion=448
Expecting property name enclosed in double quotes: line 2 column 3 (char 4): JSON parse failed for batch starti

In [ ]:
results_copy = results.copy()

In [ ]:
import re

for key,text in all_texts.items():
  try:
    fixed_text = re.sub(r'(\s*)(\d+\.\d+)\s*:', r'\1"\2":', text)
    batch_results = json.loads(fixed_text)
    results_copy.update(batch_results)
    if key == 100:
      fixed_text = re.sub(r'(\s*)(\d+)\s*:', r'\1"1.\2":', text)
      batch_results = json.loads(fixed_text)
      results_copy.update(batch_results)
      print(batch_results.keys())
  except Exception as e:
    print(e)
    print(key)

Expecting property name enclosed in double quotes: line 2 column 3 (char 4)
100


In [ ]:
with open("Dataset/Test/test_dataset.jsonl", "w", encoding="utf-8") as f:
    for item in results_copy:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

# Optionally convert to CSV
output_df = pd.DataFrame(results_copy)
output_df.T.to_csv("Dataset/Test/test_dataset.csv", encoding="utf-8")


In [ ]:
output_df.shape

(2, 820)

In [ ]:
fixed_text = re.sub(r'(\s*)(\d+)\s*:', r'\1"1.\2":', all_texts[100])
batch_results = json.loads(fixed_text)
fixed_text

'{\n  "1.180": [\n    "What is the significance of Aurva\'s conversation with his ancestors in the Mahabharata?",\n    "How did Aurva express his anger and what was the outcome of that action?"\n  ],\n  "1.183": [\n    "Who was appointed as the priest for the Pandavas and what role did he play?",\n    "What led to the selection of Dhaumya as the Pandavas\' priest?"\n  ],\n  "1.184": [\n    "What events transpired during the Pandavas\' journey to Panchal?",\n    "How did the Pandavas interact with the Brahmins on their way to Panchal?"\n  ],\n  "1.185": [\n    "What was the significance of the Swayamvar Sabha in Drupada\'s capital?",\n    "What declaration did Dhrishtadyumna make regarding Draupadi\'s swayamvara?"\n  ],\n  "1.186": [\n    "Who were the kings introduced by Dhristadyumna at Draupadi\'s swayamvara?",\n    "What role did the introduction of kings play in the context of Draupadi\'s marriage?"\n  ],\n  "1.187": [\n    "What challenges did the kings face while trying to hit th

In [ ]:
batch_results.keys()

dict_keys(['1.180', '1.183', '1.184', '1.185', '1.186', '1.187', '1.188', '1.189', '1.190', '1.191'])

In [ ]:
results_copy.update(batch_results)

In [ ]:
output_df.T

,0,1
1.1,What are the main topics covered in the Mahabh...,How does the Mahabharata highlight the signifi...
1.2,Can you explain the significance of the Samant...,What are the key parvas mentioned in the Mahab...
1.3,What led to Janamejaya receiving a curse from ...,"How do the stories of Aruni, Upamanyu, and Utt..."
1.5,What was the conversation between Puloma and A...,How did Puloma's arrival impact Agnidev in the...
1.6,What events led to the birth of Maharshi Chyav...,How did Bhrigu's curse affect Agni after the b...
...,...,...
1.187,What challenges did the kings face while tryin...,How did the kings react to their failures duri...
1.188,What was Arjuna's strategy to win Draupadi's h...,How did Arjuna manage to hit the target during...
1.189,What advice did Krishna give to Bhima and Arju...,How did Krishna and Balarama discuss the situa...
1.190,What events led to the defeat of Karna and Sha...,How did the Pandavas return to their camp afte...


In [ ]:
filtered_results = {}
for ch_id, questions in results_copy.items():
    filtered_questions = [q for q in questions if 'chapter' not in q.lower()]
    if filtered_questions:
        filtered_results[ch_id] = filtered_questions

results_copy = filtered_results

# Display a sample of the cleaned results or its size to confirm
print(f"Number of entries after filtering: {len(results_copy)}")
# If you want to see a few examples:
# for i, (ch_id, q_list) in enumerate(results_copy.items()):
#     if i >= 5: break
#     print(f'{ch_id}: {q_list}')


Number of entries after filtering: 810


In [ ]:
output_df = pd.DataFrame.from_dict(filtered_results, orient='index')
output_df.index = "M." + output_df.index.astype(str)

output_df.to_csv("Dataset/Test/test_dataset.csv", encoding="utf-8")

print(output_df.shape)

(810, 2)


In [80]:
output_df

,0,1
M.1.1,What are the main topics covered in the Mahabh...,How does the Mahabharata highlight the signifi...
M.1.2,Can you explain the significance of the Samant...,What are the key parvas mentioned in the Mahab...
M.1.3,What led to Janamejaya receiving a curse from ...,"How do the stories of Aruni, Upamanyu, and Utt..."
M.1.5,What was the conversation between Puloma and A...,How did Puloma's arrival impact Agnidev in the...
M.1.6,What events led to the birth of Maharshi Chyav...,How did Bhrigu's curse affect Agni after the b...
...,...,...
M.1.187,What challenges did the kings face while tryin...,How did the kings react to their failures duri...
M.1.188,What was Arjuna's strategy to win Draupadi's h...,How did Arjuna manage to hit the target during...
M.1.189,What advice did Krishna give to Bhima and Arju...,How did Krishna and Balarama discuss the situa...
M.1.190,What events led to the defeat of Karna and Sha...,How did the Pandavas return to their camp afte...


In [2]:
import pandas as pd

df = pd.read_csv('Dataset/Test/test_dataset.csv')

In [4]:
df

,Unnamed: 0,0,1
0,M.1.1,What are the main topics covered in the Mahabh...,How does the Mahabharata highlight the signifi...
1,M.1.2,Can you explain the significance of the Samant...,What are the key parvas mentioned in the Mahab...
2,M.1.3,What led to Janamejaya receiving a curse from ...,"How do the stories of Aruni, Upamanyu, and Utt..."
3,M.1.5,What was the conversation between Puloma and A...,How did Puloma's arrival impact Agnidev in the...
4,M.1.6,What events led to the birth of Maharshi Chyav...,How did Bhrigu's curse affect Agni after the b...
...,...,...,...
805,M.1.187,What challenges did the kings face while tryin...,How did the kings react to their failures duri...
806,M.1.188,What was Arjuna's strategy to win Draupadi's h...,How did Arjuna manage to hit the target during...
807,M.1.189,What advice did Krishna give to Bhima and Arju...,How did Krishna and Balarama discuss the situa...
808,M.1.190,What events led to the defeat of Karna and Sha...,How did the Pandavas return to their camp afte...


In [15]:
import pandas as pd
import json

# Load the CSV (ensure all as strings)
df = pd.read_csv("Dataset/Test/test_dataset.csv", dtype=str)

# Prepare result dictionary
result = {}
q_counter = 1

# Iterate through each row
for _, row in df.iterrows():
    ch_id = row["Unnamed: 0"]  # column with chapter IDs

    for col in df.columns:
        if col not in ["Unnamed: 0"]:  # skip ID column
            question_text = str(row[col]).strip()
            if pd.notna(question_text) and question_text != "nan":
                q_key = f"q{q_counter}"
                result[q_key] = {
                    "question": question_text,
                    "ground_truth": ch_id
                }
                q_counter += 1


In [17]:
len(result)

1536

In [18]:

# Save to JSON
with open("Dataset/Test/questions.json", "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

print(f"✅ Created {len(result)} query entries and saved to test_ir_dataset.json")


✅ Created 1536 query entries and saved to test_ir_dataset.json
